## 1. Prepare data

In [ ]:
import os
os.chdir("utils")
from make_txt import make_txt
os.chdir("..")

folder = "/home/user/ssd1/taeholee/SSD/coco-ssd/data/"
annotations = "/home/user/hdd/coco/annotations/"

make_txt(100, txt_folder=folder, annotations=annotations)

## 2. Train

In [ ]:
import os
import itertools
import torch
import torch.nn as nn


from torch.utils.data import DataLoader, ConcatDataset
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR

os.chdir("utils")
from misc import store_labels
os.chdir("..")

from ssd.ssd import MatchPrior
from ssd.vgg_ssd import create_vgg_ssd
from ssd.config import vgg_ssd_config
from ssd.data_preprocessing import TrainAugmentation, TestTransform

from datasets.COCO_dataset import COCODataset

from nn.multibox_loss import MultiboxLoss

from train_and_test import train, test

print('Single Shot MultiBox Detector Training With Pytorch')

dataset_type = "COCO"
datasets = "/home/user/hdd/coco/"
validation_dataset = "/home/user/hdd/coco/"
txt_folder = "/home/user/ssd1/taeholee/SSD/coco-ssd/data/"
net = "vgg16-ssd"

# Params for SGD
lr = 1e-3 #1e-3
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
base_net_lr = None
extra_layers_lr = None

# Params for loading pretrained basenet or checkpoints.
folder = "/home/user/ssd1/taeholee/SSD/coco-ssd/models/"
base_net = folder + "vgg16_reducedfc.pth"
pretrained_ssd = None
resume = folder + "vgg16-ssd-Epoch-49-Loss-4.91829.pth"

# Scheduler
scheduler = "multi-step"

# Params for Multi-step Scheduler
milestones = "120,160"

# Params for Cosine Annealing
t_max = 120.0

# Train params
batch_size = 8 #24
num_epochs = 2 #100
num_workers = 8
validation_epochs = 1
debug_steps = 100
use_cuda = True
use_multi_gpu = True
checkpoint_folder = folder
#num_training_loss = 0 #in train_and_test.py



###main
if use_cuda and torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.benchmark = True
    print("Use Cuda.")
    DEVICE_IDS = [3,1] #GPU numbers
    DEVICE = torch.device("cuda:3")
else:
    DEVICE = torch.device("cpu")
    
if net == 'vgg16-ssd':
    create_net = create_vgg_ssd
    config = vgg_ssd_config
else:
    print("No network")
    
train_transform = TrainAugmentation(config.image_size,
                                    config.image_mean,
                                    config.image_std)
target_transform = MatchPrior(config.priors,
                              config.center_variance,
                              config.size_variance, 0.5)
test_transform = TestTransform(config.image_size,
                               config.image_mean,
                               config.image_std)


print("Prepare training and validation datasets.")
datalist = []
if dataset_type == 'COCO':
    dataset     = COCODataset(root=datasets, txt=txt_folder,
                              transform=train_transform,
                              target_transform=target_transform,
                              is_test=False, is_validate=False)
    val_dataset = COCODataset(root=validation_dataset, txt=txt_folder,
                              transform=test_transform,
                              target_transform=target_transform,
                              is_test=False, is_validate=True)
    
    label_file = os.path.join(checkpoint_folder, "coco-model-labels.txt")
    store_labels(label_file, dataset.class_names)
    num_classes = len(dataset.class_names)
    print("Stored labels into file coco-model-labels.txt.")
else:
    print("No dataset_type")


datalist.append(dataset)
train_dataset = ConcatDataset(datalist)

print("Train dataset size: {}". format(len(train_dataset)))
print("validation dataset size: {}".format(len(val_dataset)))

train_loader = DataLoader(train_dataset, batch_size,
                          num_workers=num_workers,
                          shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size,
                          num_workers=num_workers,
                          shuffle=True)


print("Build network.")
net = create_net(num_classes, device=DEVICE, is_test=False)
min_loss = -10000.0
last_epoch = -1

base_net_lr = base_net_lr if base_net_lr is not None else lr
extra_layers_lr = extra_layers_lr if extra_layers_lr is not None else lr
    
params = [
            {'params': net.base_net.parameters(), 'lr': base_net_lr},
            {'params': itertools.chain(net.source_layer_add_ons.parameters(),
                                       net.extras.parameters()),
             'lr': extra_layers_lr},
            {'params': itertools.chain(net.regression_headers.parameters(),
                                       net.classification_headers.parameters())}
         ]


if resume:
    print("Resume from the model:", resume, "\n")
    torch.load(resume)
    #net.load_state_dict(resume)
    #net.load_state_dict(torch.load(resume))
elif pretrained_ssd:
    print("Init from pretrained ssd\n")
    net.init_from_pretrained_ssd(pretrained_ssd)
elif base_net:
    print("Init from base net\n")
    net.init_from_base_net(base_net)
    

if use_cuda and use_multi_gpu and torch.cuda.device_count() > 1: # using multi-GPU
    print("Let's use", tuple(DEVICE_IDS), "multi-GPU number.")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    net = nn.DataParallel(net, device_ids=DEVICE_IDS, output_device=DEVICE_IDS[0])

net.to(DEVICE)
criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3,
                         center_variance=0.1, size_variance=0.2, device=DEVICE)
optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum,
                            weight_decay=weight_decay)

print("Learning rate:", lr, "Base net learning rate:", base_net_lr,
      "Extra Layers learning rate:", extra_layers_lr)

if scheduler == 'multi-step':
    print("Uses MultiStepLR scheduler.")
    milestones = [int(v.strip()) for v in milestones.split(",")]
    scheduler = MultiStepLR(optimizer, milestones=milestones,
                            gamma=0.1, last_epoch=last_epoch)
elif scheduler == 'cosine':
    print("Uses CosineAnnealingLR scheduler.")
    scheduler = CosineAnnealingLR(optimizer, t_max, last_epoch=last_epoch)
else:
    print("Unsupported Scheduler")


print("Start training from epoch 0.\n")
for epoch in range(last_epoch + 1, num_epochs):
    scheduler.step()
    train(train_loader, net, criterion, optimizer,
          device=DEVICE, checkpoint_folder=checkpoint_folder,
          debug_steps=debug_steps, epoch=epoch)
        
    if epoch % validation_epochs == 0 or epoch == num_epochs - 1:
        val_loss, val_regression_loss, val_classification_loss = test(val_loader,
                                                                      net,
                                                                      criterion,
                                                                      DEVICE)
        print("Epoch:", epoch, "\n", 
              "Validation Loss:", val_loss, "\n",
              "Validation Regression Loss:", val_regression_loss, "\n",
              "Validation Classification Loss:", val_classification_loss
              )
        
        name = "vgg16-ssd"+"-Epoch-"+"%s"%epoch+"-Loss-"+"%s"%str(val_loss)[:7]+".pth"
        model_path = os.path.join(checkpoint_folder, name)
        torch.save(net.state_dict(), model_path)
        print("Saved model:", model_path, "\n")
        


## 3. Eval
__'difficult':__


an object marked as 'difficult', indicates that the object is considered difficult to recognize,  
for example an object which is clearly visible but unidentifiable without substantial use of context.  
Objects marked as difficult are currently ignored in the evaluation of the challenge.

In [ ]:
import os
import pathlib
import numpy as np
import torch
import torch.nn as nn

from ssd.vgg_ssd import create_vgg_ssd, create_vgg_ssd_predictor

from datasets.COCO_dataset import COCODataset

os.chdir("utils")
from compute_average_precision_per_class import compute_average_precision_per_class
from group_annotation_by_class import group_annotation_by_class
os.chdir("..")


print("SSD Evaluation on COCO Dataset.")

net = "vgg16-ssd"
folder = "/home/user/ssd1/taeholee/SSD/coco-ssd/"
trained_model = folder + "models/vgg16-ssd-Epoch-3-Loss-20.2706.pth"
label_file = folder + "models/coco-model-labels.txt"
dataset_type = "COCO"
dataset = "/home/user/hdd/coco/"
txt_folder = folder + "data/"

use_cuda = True
use_2007_metric = True # figure out: It computes average precision based on the definition of Pascal Competition.
nms_method = "hard"
iou_threshold = 0.5
eval_dir = "eval_results"


if use_cuda and torch.cuda.is_available():
    print("Use Cuda.")
    DEVICE = torch.device("cuda:3")
else:
    DEVICE = torch.device("cpu")
    
if use_2007_metric:
    print("Computes average precision based on the definition of Pascal Competition.")

#main    
eval_path = pathlib.Path(eval_dir)
eval_path.mkdir(exist_ok=True)
    
class_names = [name.strip() for name in open(label_file).readlines()]

if dataset_type == "COCO":
    dataset = COCODataset(root=dataset, txt=txt_folder, is_test=True)
else:
    print("No dataset_type")

print("Test dataset size: {}". format(len(dataset)))
true_case_stat, all_gb_boxes, all_difficult_cases = group_annotation_by_class(dataset)
    
if net == 'vgg16-ssd':
    net = create_vgg_ssd(len(class_names), device=DEVICE, is_test=True)
    predictor = create_vgg_ssd_predictor(net, nms_method=nms_method, device=DEVICE)
else:
    print("No net type")

print("\nLoad Model:", trained_model)
torch.load(trained_model)
#net.load(trained_model)
net.to(DEVICE)


results = []
for i in range(len(dataset)):
    if i == 0:
        print("process image", i, "start")
    if i % 1000 == 0 and i != 0:
        print("process image", i)
    if i == (len(dataset)-1):
        print("process image", i, "end")
    
    image = dataset.get_image(i)
    boxes, labels, probs = predictor.predict(image)
    indexes = torch.ones(labels.size(0), 1, dtype=torch.float32) * i
    results.append(torch.cat([indexes.reshape(-1, 1),
                              labels.reshape(-1, 1).float(),
                              probs.reshape(-1, 1),
                              boxes + 0.0  # matlab's indexes start from 1
                             ], dim=1))
        
results = torch.cat(results)
    
aps = []
print("\nAverage Precision Per-class:\n")
for class_index, class_name in enumerate(class_names):
    if class_index == 0:
        continue  # ignore background
        
    prediction_path = str(eval_path) + "/det_test_%s.txt" %class_name
        
    with open(prediction_path, "w") as f:
        sub = results[results[:, 1] == class_index, :]
        for i in range(sub.size(0)):
            prob_box = sub[i, 2:].numpy()
            image_id = dataset.ids[int(sub[i, 0])]
            print(image_id + " " + " ".join([str(v) for v in prob_box]), file=f)
        
    try:
        ap = compute_average_precision_per_class(true_case_stat[class_index],
                                                 all_gb_boxes[class_index],
                                                 all_difficult_cases[class_index],
                                                 prediction_path,
                                                 iou_threshold,
                                                 use_2007_metric)
    except KeyError:
        pass
        
    aps.append(ap)
    if ap > 1e-7: #exclude small ap
        print("%s:" %class_name, "%f" %ap)
    
print("\nAverage Precision Across All Classes: %f" %(sum(aps)/len(aps)))
    


## 4. Visualize

In [ ]:
import os
os.chdir('utils')
from image_print import image_print
os.chdir('..')
%matplotlib inline


net_type = "vgg16-ssd"
folder = "/home/user/ssd1/taeholee/SSD/coco-ssd/"
model_path = folder + "models/vgg16-ssd-Epoch-3-Loss-20.2706.pth"
label_path = folder + "models/coco-model-labels.txt"
data_dir = "/home/user/hdd/coco/"
test_txt = folder + "data/test.txt"
how_many_images = 2

print("Use model:", model_path, "\n")
image_print(net_type=net_type,
            model_path=model_path,
            label_path=label_path,
            data_dir=data_dir,
            test_txt=test_txt,
            how_many_images=how_many_images)
